# Solving FetchReach with PPO


In [ ]:
#
# Installing Stable Baselines3
#
!pip install stable-baselines3 gymnasium-robotics mujoco -q
!apt-get install -y xvfb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import gymnasium as gym
import gymnasium_robotics
import numpy as np

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CheckpointCallback

from gym.wrappers import RecordVideo
from IPython.display import Video
import glob

In [ ]:
print(gym.envs.registry.keys())
print(gym.envs.registry["FetchReach-v3"])

dict_keys(['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'phys2d/CartPole-v0', 'phys2d/CartPole-v1', 'phys2d/Pendulum-v0', 'LunarLander-v3', 'LunarLanderContinuous-v3', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v3', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'tabular/Blackjack-v0', 'tabular/CliffWalking-v0', 'Reacher-v2', 'Reacher-v4', 'Reacher-v5', 'Pusher-v2', 'Pusher-v4', 'Pusher-v5', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedPendulum-v5', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'InvertedDoublePendulum-v5', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'HalfCheetah-v5', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4', 'Hopper-v5', 'Swimmer-v2', 'Swimmer-v3', 'Swimmer-v4', 'Swimmer-v5', 'Walker2d-v2', 'Walker2d-v3', 'Walker2d-v4', 'Walker2d-v5', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Ant-v5', 'Humanoid-v2', 'Humanoid-v3', 'Humanoid-v4', 'Humanoid-v5

In [ ]:
# Enable EGL rendering
os.environ['MUJOCO_GL'] = 'egl'

# Create and wrap environment
env = gym.make("FetchReach-v3", reward_type="dense", render_mode="rgb_array")

In [ ]:
# Set up callback for saving checkpoints
checkpoint_callback = CheckpointCallback(
    save_freq=50_000,
    save_path='./ppo_checkpoints/',
    name_prefix='ppo_model'
)

In [ ]:
#
# Train the model
#
total_timesteps = 300_000
model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=total_timesteps, callback=checkpoint_callback)

# Save the model
model.save("/content/drive/MyDrive/fetchreach_PPO_model_300k") # it's a zip file
env.close()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | -27.1    |
|    success_rate    | 0        |
| time/              |          |
|    fps             | 243      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 50           |
|    ep_rew_mean          | -26.6        |
|    success_rate         | 0            |
| time/                   |              |
|    fps                  | 212          |
|    iterations           | 2            |
|    time_elapsed         | 19           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0050321147 |
|    clip_fract

In [ ]:
#
# Evaluate the trained model
#
# Load back model (if necessary)
model = PPO.load("/content/drive/MyDrive/fetchreach_PPO_model_250k")
env = gym.make("FetchReach-v3", reward_type="dense", render_mode=None)

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(f"\n[Evaluate_Policy] Mean reward: {mean_reward:.2f} ± {std_reward:.2f}")

env.close()

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(



[Evaluate_Policy] Mean reward: -8.08 ± 2.53


In [ ]:
# Manual evaluation over 100 episodes with success rate
env = gym.make("FetchReach-v3", reward_type="dense", render_mode=None)
manual_rewards = []

for ep in range(100):
    obs, _ = env.reset()
    done = False
    episode_reward = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        episode_reward += reward
    manual_rewards.append(episode_reward)

manual_mean = np.mean(manual_rewards)
manual_std = np.std(manual_rewards)

print(f"[Manual Evaluation] Mean reward: {manual_mean:.2f} ± {manual_std:.2f}")

# Comparison
print("\n--- Comparison ---")
print(f"evaluate_policy(): Mean = {mean_reward:.2f}, Std = {std_reward:.2f}")
print(f"Manual rollout:   Mean = {manual_mean:.2f}, Std = {manual_std:.2f}")

env.close()

[Manual Evaluation] Mean reward: -8.24 ± 2.68

--- Comparison ---
evaluate_policy(): Mean = -8.08, Std = 2.53
Manual rollout:   Mean = -8.24, Std = 2.68


**Video Generation**

In [ ]:
# Simulate one trajectory with video recording
env = gym.make("FetchReach-v3", reward_type="dense", render_mode="rgb_array")
env = RecordVideo(env, video_folder="video", episode_trigger=lambda x: True)

obs, _ = env.reset()
achieved, desired = [], []
total_reward = 0
min_distance = float("inf")

for _ in range(100):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    total_reward += reward

    # Track goals
    ag = obs["achieved_goal"]
    dg = obs["desired_goal"]
    achieved.append(ag)
    desired.append(dg)

    # Compute distance
    dist = np.linalg.norm(ag - dg)
    min_distance = min(min_distance, dist)

    if done:
        break

env.close()

achieved = np.array(achieved)
desired = np.array(desired)
total_dist = np.sum(np.linalg.norm(achieved - desired, axis=1))

print(f"Trajectory total reward = {total_reward:.4f}")
print(f"Total Euclidean distance = {total_dist:.4f}")
print(f"Match: {np.isclose(total_reward, -total_dist)}")
print(f"Minimum distance to goal during trajectory = {min_distance:.4f}")


/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /content/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Trajectory total reward = -9.5808
Total Euclidean distance = 9.5808
Match: True
Minimum distance to goal during trajectory = 0.0827


In [ ]:
# Display video
mp4list = glob.glob('video/*.mp4')
mp4list.sort()
Video(mp4list[-1], embed=True)